In [5]:
from pandas_datareader import data as pdr
import pandas as pd
import numpy as np
import cvxpy as cvx
from scipy.stats import norm
from numpy.linalg import inv
import math

import keras
import seaborn as sns
import sklearn
import tensorflow as tf
import matplotlib.pyplot as plt

# for plotting
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import matplotlib.pyplot as plt

# Initialize plotly offline
plotly.offline.init_notebook_mode(connected=True)

# Helper Functions

## Data functions

In [6]:
# Download Stock data from Yahoo Finance
def download_stock_data(start_date, end_date, asset_list, filename):
    """
    asset_list: list of asset to download
    filename: name of the csv file where the data go to
    """
    for i, asset in enumerate(asset_list):
        if i == 0:
            data = pdr.get_data_yahoo(asset_list[i], start_date,end_date)[['Adj Close']]
            all_prices = data.rename(columns={"Adj Close": asset})
            
        else:
            new_asset = pdr.get_data_yahoo(asset_list[i], start_date,end_date)[['Adj Close']]
            all_prices = all_prices.join(new_asset.rename(columns={"Adj Close": asset}))
    
    all_prices.to_csv('./00_Data/' + filename + '.csv')
    return

# Extract Return and accumulated price data (starting value 1) from raw stock prices
def process_stock_data(all_prices):
    """
    input:
    all_prices: data frame, price of a list of stock from start date to the end date
    
    return:
    all_price_norm: normalized asset price with the price on the first day as 1
    all_returns: relative price
    """
    n_time, n_assets = all_prices.shape
    # Calculate returns
    all_returns = all_prices.pct_change(1).apply(lambda a : a + 1)
    
    # Calculate relative price time series
    all_prices_norm = all_returns.copy()
    for i in range(n_time):
        if i == 0:
            all_prices_norm.iloc[i] = np.ones(n_assets) # all 1 on day 1
        else:
            for j in range(n_assets):
                if math.isnan(all_prices_norm.iloc[i,j]): # if NaN, use previous day's relative price
                    all_prices_norm.iloc[i,j] = all_prices_norm.iloc[i-1,j]
                else:
                    all_prices_norm.iloc[i,j] = all_prices_norm.iloc[i,j] * all_prices_norm.iloc[i-1,j]
                    
    return all_prices_norm, all_returns.dropna()

## Test stocks

In [7]:
#1/fluc, 1, 1/fluc, 1, ...
def teststock1(T, fluc):
    stock = np.ones(T+1)
    for i in range(T+1):
        if i%2 == 1:
            stock[i] = 1.0/fluc
    return stock

#Brownian Motion
def teststock2(T, delta, sigma):
    stock1 = np.zeros(T+1)
    stock1[0] = 3.0
    for i in range(T):
        stock1[i+1] = stock1[i] + sigma + norm.rvs(scale=delta**2)
        if (stock1[i+1] <= 0):
            print('Negative Warning!')
    return stock1

#Garbage 1, 1/fluc, 1/pow(fluc,2) ,...
def teststock3(T, discount):
    stock = np.ones(T+1, dtype=float)
    for i in range(T):
        stock[i+1] = stock[i]*discount
    return stock

## Risk Aware Portfolio Selection Algorithm
#### Bug: cannot initialize gamma as cp.Parameter

In [8]:
def Risk_Aware_Portfolio(stock_returns, gamma, delta):
    """
    input:
    stock_return: returns for the stock data
    gamma: the confidence level
    delta: number of dates we use to estimate the density of returns
    
    output:
    portfolio: portfolio selected
    """
    u = cp.Variable((stock_returns.shape[1], 1))
    a = cp.Variable()
    #     gamma = cp.Parameter(gamma)

    constraints = [cp.sum(u) == 1, u >= 0]

    portfolio = []

    for t in range(2, len(stock_returns)):
        coefficient = 1 / ((delta + t - 1) * (1 - gamma))

        sliced = stock_returns.iloc[:delta]
        sliced = sliced.to_numpy()
        inner_part = cp.pos(sliced * (-u) - a)
        first_sum = cp.sum(inner_part)

        sliced2 = stock_returns.iloc[:t - 1]
        sliced2 = sliced2.to_numpy()
        inner_part2 = sliced2 * (-u) - a
        second_sum = cp.sum(cp.pos(inner_part2))

        obj = cp.Minimize(a + coefficient * (first_sum + second_sum))

        prob = cp.Problem(obj, constraints)
        prob.solve(solver='SCS')
        portfolio.append(u.value)

    return portfolio

    """
    n_time, n_assets = stock_returns.shape
    portfolio = np.zeros((n_assets, n_time)) # the output portfolio
    u = cp.Variable(n_assets) # portfolio vector
    alpha = cp.Variable() # auxiliary variable
    
    constraints = [cp.sum(u) == 1, 
                u >= 0]
    Gamma = cp.Parameter(nonneg = True,value = gamma)
    
    for t in range(delta + 1, n_time):
        R = stock_returns.iloc[:t,:].dot(u).values
        fct =  alpha + (cp.pos(-cp.sum(R[:delta])-alpha)+cp.pos(-cp.sum(R[:t-1])-alpha))/((delta + t - 1)*(1 - Gamma))
        prob = cp.Problem(cp.Minimize(fct), constraints)
        p = prob.solve()
        portfolio[:,t] = u.value
        
    return portfolio
    """

## Normalization

In [9]:
def normalized(data):
    data_temp = data - np.mean(data)
    variance = np.sum(data ** 2)/len(data)
    data_temp /= variance
    return data_temp

## Load stock data

### US Stocks

In [10]:
us_stocks = ['SPY', # The SP500 as a benchmark
             'AAPL', 'MMM', 'BA' , 'CAT', 'CVX' , 'CSCO', 'KO' , 'DIS',
             'GS', 'IBM', 'INTC', 'JNJ', 'JPM', 'MSFT', 'PFE'] #Some stocks from the Dow Jones

start_data = '2008-09-01'
end_date = '2008-10-01'

In [11]:
# download_stock_data(start_data, end_date, us_stocks, 'us_stocks_latest')

In [12]:
# read in the data again
us_stock_prices = pd.read_csv('./00_Data/us_stocks_latest.csv', header=0, index_col = 0)
us_stock_prices.index = pd.to_datetime(us_stock_prices.index)

#Process Stock Data
stock_prices_norm, stock_returns = process_stock_data(us_stock_prices)

# Extract benchmark SP500 Data
SPY_relative_price_benchmark = stock_prices_norm.drop(us_stocks[1:], axis=1)
SPY_returns_benchmark = stock_returns.drop(us_stocks[1:], axis=1)
us_stock_prices_norm_wo_SPY = stock_prices_norm.drop(us_stocks[0], axis=1)

# filter out benchmark data from stock returns
us_stock_returns = stock_returns.drop(us_stocks[0], axis=1)

### Hundred stocks

In [13]:
# Read in the data
hundred_stock_prices = pd.read_csv('./00_Data/hundred_stocks.csv', header=0, index_col = 0)

In [14]:
hundred_stock_prices.index = pd.to_datetime(hundred_stock_prices.index)

#Process Stock Data
hundred_stock_prices_norm, hundred_stock_returns = process_stock_data(hundred_stock_prices)

### Subprime mortgage crisis period data

In [15]:
subprime_mortgage_crisis = hundred_stock_prices.loc['2008-09'].dropna(axis=1) 
SPMC_relative_prices, SPMC_returns = process_stock_data(subprime_mortgage_crisis) # H_i,t of each asset i for t = 1,...,20

In [16]:
SPMC_SPY_relative_price_benchmark = SPY_relative_price_benchmark.loc['2008-09']

# Directions: Deep portfolio using neural network

Next we are to implement the deep portfolio. Recall the four step procedure: 
- **Auto-encoding**: this phase is essentially dimension reduction. *In the paper: one hidden layer with five neurons*.
- **Calibration**: using the stock price of the first few years, our target index $Y$ is the S&P500 (we can also play with this target so that the portfolio map can possibly outperform S&P500 -- but this is for later). *In the paper: ReLU and 4-fold cross validation*. 
- **Validation**: out-sample validation. Will need to test with different number of stocks.
- **Verification**: deep frontier. We look at the 2-norm out-of-sample error for different number of stocks used.

## 1. Auto-encoding

**Used Data**: 
- SPMC_returns: 20 daily returns during September of 2008 calculated with hundred_stocks.

**NN Model**: 
- 87 - 5 - 87

In [17]:
# Import `Sequential` from `keras.models`
from keras.models import Sequential
from keras.optimizers import RMSprop
opt = RMSprop(lr=0.001)

# Import `Dense` from `keras.layers`
from keras.layers import Dense

### Constructing model

In [18]:
# Initialize the constructor
autoencoder_model = Sequential()

#input 87 stock data, and the hidden layer has 5 neurons; no biased involved (unless we let use_bias = TRUE)
autoencoder_model.add(Dense(units = 5, input_dim = 87, activation='relu', use_bias = False))
autoencoder_model.add(Dense(units = 87, activation='relu', use_bias = False))

W0731 23:52:03.288613 4460418496 deprecation_wrapper.py:119] From /usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0731 23:52:03.290719 4460418496 deprecation_wrapper.py:119] From /usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0731 23:52:03.293617 4460418496 deprecation_wrapper.py:119] From /usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



### Set optimizer and learning rate

In [19]:
# autoencoder_opt = RMSprop(lr=0.001)

### Compile & Learn

In [20]:
autoencoder_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

W0731 23:57:27.970893 4460418496 deprecation_wrapper.py:119] From /usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [21]:
autoencoder_model.fit(SPMC_returns, SPMC_returns, epochs = 500)

W0731 23:57:28.933850 4460418496 deprecation_wrapper.py:119] From /usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0731 23:57:28.970160 4460418496 deprecation_wrapper.py:119] From /usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/500
20/20 [==============================] - 0s 8ms/step - loss: 2.3359 - mean_absolute_error: 0.9791
Epoch 2/500
20/20 [==============================] - 0s 61us/step - loss: 2.3176 - mean_absolute_error: 0.9719
Epoch 3/500
20/20 [==============================] - 0s 54us/step - loss: 2.2997 - mean_absolute_error: 0.9648
Epoch 4/500
20/20 [==============================] - 0s 68us/step - loss: 2.2837 - mean_absolute_error: 0.9583
Epoch 5/500
20/20 [==============================] - 0s 64us/step - loss: 2.2663 - mean_absolute_error: 0.9512
Epoch 6/500
20/20 [==============================] - 0s 67us/step - loss: 2.2490 - mean_absolute_error: 0.9439
Epoch 7/500
20/20 [==============================] - 0s 72us/step - loss: 2.2320 - mean_absolute_error: 0.9367
Epoch 8/500
20/20 [==============================] - 0s 64us/step - loss: 2.2151 - mean_absolute_error: 0.9293
Epoch 9/500
20/20 [==============================] - 0s 56us/step - loss: 2.1985 - mean_absolute_error: 0.9218
Ep

Epoch 75/500
20/20 [==============================] - 0s 53us/step - loss: 1.5654 - mean_absolute_error: 0.5503
Epoch 76/500
20/20 [==============================] - 0s 48us/step - loss: 1.5602 - mean_absolute_error: 0.5478
Epoch 77/500
20/20 [==============================] - 0s 67us/step - loss: 1.5551 - mean_absolute_error: 0.5454
Epoch 78/500
20/20 [==============================] - 0s 54us/step - loss: 1.5500 - mean_absolute_error: 0.5430
Epoch 79/500
20/20 [==============================] - 0s 51us/step - loss: 1.5449 - mean_absolute_error: 0.5407
Epoch 80/500
20/20 [==============================] - 0s 48us/step - loss: 1.5399 - mean_absolute_error: 0.5385
Epoch 81/500
20/20 [==============================] - 0s 43us/step - loss: 1.5350 - mean_absolute_error: 0.5363
Epoch 82/500
20/20 [==============================] - 0s 58us/step - loss: 1.5301 - mean_absolute_error: 0.5343
Epoch 83/500
20/20 [==============================] - 0s 47us/step - loss: 1.5254 - mean_absolute_error:

Epoch 148/500
20/20 [==============================] - 0s 81us/step - loss: 1.3407 - mean_absolute_error: 0.5055
Epoch 149/500
20/20 [==============================] - 0s 70us/step - loss: 1.3386 - mean_absolute_error: 0.5057
Epoch 150/500
20/20 [==============================] - 0s 97us/step - loss: 1.3366 - mean_absolute_error: 0.5058
Epoch 151/500
20/20 [==============================] - 0s 104us/step - loss: 1.3345 - mean_absolute_error: 0.5059
Epoch 152/500
20/20 [==============================] - 0s 84us/step - loss: 1.3325 - mean_absolute_error: 0.5060
Epoch 153/500
20/20 [==============================] - 0s 70us/step - loss: 1.3305 - mean_absolute_error: 0.5062
Epoch 154/500
20/20 [==============================] - 0s 59us/step - loss: 1.3285 - mean_absolute_error: 0.5063
Epoch 155/500
20/20 [==============================] - 0s 54us/step - loss: 1.3264 - mean_absolute_error: 0.5064
Epoch 156/500
20/20 [==============================] - 0s 54us/step - loss: 1.3244 - mean_absol

Epoch 221/500
20/20 [==============================] - 0s 103us/step - loss: 1.1834 - mean_absolute_error: 0.4995
Epoch 222/500
20/20 [==============================] - 0s 60us/step - loss: 1.1810 - mean_absolute_error: 0.4992
Epoch 223/500
20/20 [==============================] - 0s 67us/step - loss: 1.1785 - mean_absolute_error: 0.4989
Epoch 224/500
20/20 [==============================] - 0s 119us/step - loss: 1.1761 - mean_absolute_error: 0.4986
Epoch 225/500
20/20 [==============================] - 0s 82us/step - loss: 1.1736 - mean_absolute_error: 0.4983
Epoch 226/500
20/20 [==============================] - 0s 105us/step - loss: 1.1711 - mean_absolute_error: 0.4980
Epoch 227/500
20/20 [==============================] - 0s 70us/step - loss: 1.1686 - mean_absolute_error: 0.4977
Epoch 228/500
20/20 [==============================] - 0s 72us/step - loss: 1.1661 - mean_absolute_error: 0.4974
Epoch 229/500
20/20 [==============================] - 0s 69us/step - loss: 1.1636 - mean_abs

Epoch 294/500
20/20 [==============================] - 0s 73us/step - loss: 0.9543 - mean_absolute_error: 0.4436
Epoch 295/500
20/20 [==============================] - 0s 83us/step - loss: 0.9507 - mean_absolute_error: 0.4421
Epoch 296/500
20/20 [==============================] - 0s 56us/step - loss: 0.9472 - mean_absolute_error: 0.4407
Epoch 297/500
20/20 [==============================] - 0s 73us/step - loss: 0.9438 - mean_absolute_error: 0.4393
Epoch 298/500
20/20 [==============================] - 0s 89us/step - loss: 0.9404 - mean_absolute_error: 0.4379
Epoch 299/500
20/20 [==============================] - 0s 54us/step - loss: 0.9371 - mean_absolute_error: 0.4365
Epoch 300/500
20/20 [==============================] - 0s 47us/step - loss: 0.9339 - mean_absolute_error: 0.4352
Epoch 301/500
20/20 [==============================] - 0s 54us/step - loss: 0.9307 - mean_absolute_error: 0.4340
Epoch 302/500
20/20 [==============================] - 0s 103us/step - loss: 0.9275 - mean_absol

Epoch 367/500
20/20 [==============================] - 0s 62us/step - loss: 0.7279 - mean_absolute_error: 0.3909
Epoch 368/500
20/20 [==============================] - 0s 43us/step - loss: 0.7251 - mean_absolute_error: 0.3907
Epoch 369/500
20/20 [==============================] - 0s 62us/step - loss: 0.7223 - mean_absolute_error: 0.3904
Epoch 370/500
20/20 [==============================] - 0s 126us/step - loss: 0.7196 - mean_absolute_error: 0.3902
Epoch 371/500
20/20 [==============================] - 0s 54us/step - loss: 0.7168 - mean_absolute_error: 0.3899
Epoch 372/500
20/20 [==============================] - 0s 67us/step - loss: 0.7140 - mean_absolute_error: 0.3897
Epoch 373/500
20/20 [==============================] - 0s 49us/step - loss: 0.7112 - mean_absolute_error: 0.3894
Epoch 374/500
20/20 [==============================] - 0s 83us/step - loss: 0.7085 - mean_absolute_error: 0.3892
Epoch 375/500
20/20 [==============================] - 0s 94us/step - loss: 0.7057 - mean_absol

Epoch 440/500
20/20 [==============================] - 0s 87us/step - loss: 0.5006 - mean_absolute_error: 0.3471
Epoch 441/500
20/20 [==============================] - 0s 67us/step - loss: 0.4982 - mean_absolute_error: 0.3469
Epoch 442/500
20/20 [==============================] - 0s 63us/step - loss: 0.4957 - mean_absolute_error: 0.3467
Epoch 443/500
20/20 [==============================] - 0s 76us/step - loss: 0.4933 - mean_absolute_error: 0.3465
Epoch 444/500
20/20 [==============================] - 0s 57us/step - loss: 0.4909 - mean_absolute_error: 0.3462
Epoch 445/500
20/20 [==============================] - 0s 162us/step - loss: 0.4885 - mean_absolute_error: 0.3460
Epoch 446/500
20/20 [==============================] - 0s 60us/step - loss: 0.4862 - mean_absolute_error: 0.3457
Epoch 447/500
20/20 [==============================] - 0s 73us/step - loss: 0.4840 - mean_absolute_error: 0.3459
Epoch 448/500
20/20 [==============================] - 0s 71us/step - loss: 0.4818 - mean_absol

### Predict

In [22]:
autoencoder_result = autoencoder_model.predict(SPMC_returns)

### Visualize

In [23]:
pd.DataFrame(autoencoder_result)

,0,1,2,3,4,5,6,7,8,9,...,77,78,79,80,81,82,83,84,85,86
0,1.007775,0.972712,0.980335,0.986981,0.971901,0.0,0.985440,0.0,1.000717,1.006635,...,0.0,0.0000,0.997709,0.983173,0.0,0.977855,0.986118,0.970256,0.0,0.981537
1,0.920173,0.992785,0.966235,0.936564,0.964722,0.0,1.010326,0.0,0.949984,0.944499,...,0.0,0.9707,0.967282,1.000936,0.0,0.973553,0.987031,1.065870,0.0,0.977573
2,1.036918,1.000807,1.008262,1.014621,1.001419,0.0,1.014599,0.0,1.028761,1.035446,...,0.0,0.0000,1.026579,1.012407,0.0,1.006270,1.015851,0.999355,0.0,1.010220
3,0.980058,0.946635,0.949443,0.950294,0.961457,0.0,0.966712,0.0,0.963731,0.975923,...,0.0,0.0000,0.971032,0.965986,0.0,0.953282,0.973223,0.956804,0.0,0.958637
4,0.967245,0.933391,0.941076,0.947894,0.931574,0.0,0.945067,0.0,0.961071,0.966323,...,0.0,0.0000,0.957448,0.942799,0.0,0.938249,0.945348,0.930056,0.0,0.941659
5,1.014109,0.980359,0.981150,0.979482,1.002121,0.0,1.004404,0.0,0.993423,1.008690,...,0.0,0.0000,1.005386,1.004236,0.0,0.987803,1.013478,0.996559,0.0,0.994095
6,1.027207,0.993195,0.994553,0.993556,1.012989,0.0,1.016491,0.0,1.007668,1.022205,...,0.0,0.0000,1.018409,1.016096,0.0,1.000445,1.024804,1.008045,0.0,1.006572
7,1.035906,1.000578,1.003546,1.004446,1.016245,0.0,1.021800,0.0,1.018648,1.031535,...,0.0,0.0000,1.026365,1.021031,0.0,1.007604,1.028681,1.011326,0.0,1.013265
8,0.970815,0.937138,0.944047,0.949921,0.937779,0.0,0.950108,0.0,0.963161,0.969448,...,0.0,0.0000,0.961209,0.948057,0.0,0.942235,0.951289,0.935952,0.0,0.945945
9,1.053299,1.016609,1.023983,1.030199,1.017958,0.0,1.030971,0.0,1.044565,1.051656,...,0.0,0.0000,1.042811,1.028815,0.0,1.022244,1.032520,1.015682,0.0,1.026337


### Percentage error

In [24]:
pd.DataFrame(abs(autoencoder_result - SPMC_returns)/SPMC_returns) * 100

,BAC,CHK,GE,AMD,NOK,CSX,TEVA,ERIC,F,WFC,...,BX,GLW,SCHW,NUS,TXT,CX,OXY,CY,NEM,FLEX
Date,,,,,,,,,,,,,,,,,,,,,
2008-09-03,2.188336,2.556507,2.103730,1.747326,0.990129,100.0,3.445694,100.0,1.242156,1.312768,...,100.0,100.00000,0.352703,0.160259,100.0,2.541231,1.597213,3.585463,100.0,4.220994
2008-09-04,0.885971,0.389966,0.341767,1.372092,1.747709,100.0,1.285634,100.0,1.106386,1.284412,...,100.0,0.29864,0.603137,0.814099,100.0,0.820273,1.251559,5.820501,100.0,0.775922
2008-09-05,1.552336,0.802954,0.175232,0.580746,8.349490,100.0,2.227896,100.0,2.409525,1.533064,...,100.0,100.00000,0.399371,2.036496,100.0,0.280053,2.028767,0.106893,100.0,2.482526
2008-09-08,9.049030,1.746709,9.004957,6.692685,0.325529,100.0,1.800826,100.0,6.592267,9.270589,...,100.0,100.00000,7.247613,6.345841,100.0,9.104608,1.316569,3.224070,100.0,6.830319
2008-09-09,3.297645,2.531391,2.646228,0.282176,6.039471,100.0,4.369918,100.0,0.616484,4.041684,...,100.0,100.00000,2.796273,3.917869,100.0,2.177673,2.763958,2.306143,100.0,0.226195
2008-09-10,1.786533,5.040711,1.780232,2.221229,1.437243,100.0,0.679713,100.0,2.213390,0.817472,...,100.0,100.00000,0.729221,1.112288,100.0,1.364575,4.005419,4.219179,100.0,0.094702
2008-09-11,0.669996,0.705226,0.791890,0.573595,1.453517,100.0,0.538351,100.0,3.754783,4.272035,...,100.0,100.00000,1.110451,1.849236,100.0,1.533975,2.480400,0.220363,100.0,3.088785
2008-09-12,1.502789,4.914291,5.644321,0.254141,1.427668,100.0,2.399332,100.0,2.906882,1.829850,...,100.0,100.00000,4.905940,1.862905,100.0,8.573384,0.822968,0.243819,100.0,0.005852
2008-09-15,23.372107,3.214625,2.655511,5.444870,3.560015,100.0,3.391790,100.0,0.229568,7.233442,...,100.0,100.00000,1.498544,3.143211,100.0,1.496493,1.877688,5.115794,100.0,2.991822


## 2. Calibration
#### Note: Autoencoder used daily returns; using relative prices here.
**Inputted Data**: 
- autoencoder_result: The autoencoder's representation of SPMC_returns.
- SPMC_relative_prices: 21 days of relative prices during September of 2008 calculated with hundred_stocks.
- SPMC_SPY_relative_price_benchmark: 21 days of relative prices during September of 2008.

**Important variables**:
- top10_bot10: 20 stocks that the autoencoder did the best/worst with predicting.

**NN Model**: 
- 10 - 3 - 1

### Sort & pick stocks

In [25]:
# Error
autoencoder_prediction_error_norm2 = np.linalg.norm(abs(autoencoder_result - SPMC_returns.values), axis = 0) # Norm2 across columns
sorted_autoencoder_prediction_error_norm2 = np.sort(autoencoder_prediction_error_norm2).tolist()
sorted_stocks = [autoencoder_prediction_error_norm2.tolist().index(x) for x in sorted_autoencoder_prediction_error_norm2] # Find corresponding stocks
top10_bot10_index = sorted_stocks[:5] + sorted_stocks[-5:] # Represent stocks as original indices from norm2


In [26]:
top10_bot10 = SPMC_relative_prices.iloc[:,top10_bot10_index]

### Constructing model

In [31]:
dfp_model = Sequential()

#input 87 stock data, and the hidden layer has 5 neurons; no biased involved (unless we let use_bias = TRUE)
dfp_model.add(Dense(units = 3, input_dim = 10, activation='relu', use_bias = False))
dfp_model.add(Dense(units = 1, activation='relu', use_bias = False))


### Set optimizer and learning rate

In [28]:
# dfp_opt = RMSprop(lr=0.0001) # Seems like adam is resolving problem of getting stuck at 2.0 MAE

### Compile & learn

In [32]:
dfp_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [49]:
dfp_model.fit(top10_bot10, SPMC_SPY_relative_price_benchmark, epochs = 10000, verbose = 1)

Epoch 1/10000
21/21 [==============================] - 0s 51us/step - loss: 4.9020e-04 - mean_absolute_error: 0.0182
Epoch 2/10000
21/21 [==============================] - 0s 70us/step - loss: 4.8999e-04 - mean_absolute_error: 0.0182
Epoch 3/10000
21/21 [==============================] - 0s 45us/step - loss: 4.8978e-04 - mean_absolute_error: 0.0182
Epoch 4/10000
21/21 [==============================] - 0s 40us/step - loss: 4.8957e-04 - mean_absolute_error: 0.0182
Epoch 5/10000
21/21 [==============================] - 0s 49us/step - loss: 4.8935e-04 - mean_absolute_error: 0.0182
Epoch 6/10000
21/21 [==============================] - 0s 42us/step - loss: 4.8914e-04 - mean_absolute_error: 0.0182
Epoch 7/10000
21/21 [==============================] - 0s 40us/step - loss: 4.8892e-04 - mean_absolute_error: 0.0182
Epoch 8/10000
21/21 [==============================] - 0s 62us/step - loss: 4.8871e-04 - mean_absolute_error: 0.0182
Epoch 9/10000
21/21 [==============================] - 0s 134us/

Epoch 71/10000
21/21 [==============================] - 0s 40us/step - loss: 4.7548e-04 - mean_absolute_error: 0.0179
Epoch 72/10000
21/21 [==============================] - 0s 38us/step - loss: 4.7527e-04 - mean_absolute_error: 0.0179
Epoch 73/10000
21/21 [==============================] - 0s 44us/step - loss: 4.7507e-04 - mean_absolute_error: 0.0179
Epoch 74/10000
21/21 [==============================] - 0s 46us/step - loss: 4.7486e-04 - mean_absolute_error: 0.0179
Epoch 75/10000
21/21 [==============================] - 0s 41us/step - loss: 4.7465e-04 - mean_absolute_error: 0.0179
Epoch 76/10000
21/21 [==============================] - 0s 40us/step - loss: 4.7445e-04 - mean_absolute_error: 0.0179
Epoch 77/10000
21/21 [==============================] - 0s 39us/step - loss: 4.7424e-04 - mean_absolute_error: 0.0179
Epoch 78/10000
21/21 [==============================] - 0s 50us/step - loss: 4.7404e-04 - mean_absolute_error: 0.0179
Epoch 79/10000
21/21 [==============================] - 

21/21 [==============================] - 0s 56us/step - loss: 4.6142e-04 - mean_absolute_error: 0.0176
Epoch 141/10000
21/21 [==============================] - 0s 78us/step - loss: 4.6122e-04 - mean_absolute_error: 0.0176
Epoch 142/10000
21/21 [==============================] - 0s 102us/step - loss: 4.6102e-04 - mean_absolute_error: 0.0176
Epoch 143/10000
21/21 [==============================] - 0s 47us/step - loss: 4.6082e-04 - mean_absolute_error: 0.0175
Epoch 144/10000
21/21 [==============================] - 0s 40us/step - loss: 4.6062e-04 - mean_absolute_error: 0.0175
Epoch 145/10000
21/21 [==============================] - 0s 48us/step - loss: 4.6042e-04 - mean_absolute_error: 0.0175
Epoch 146/10000
21/21 [==============================] - 0s 56us/step - loss: 4.6022e-04 - mean_absolute_error: 0.0175
Epoch 147/10000
21/21 [==============================] - 0s 52us/step - loss: 4.6002e-04 - mean_absolute_error: 0.0175
Epoch 148/10000
21/21 [==============================] - 0s 47u

Epoch 209/10000
21/21 [==============================] - 0s 71us/step - loss: 4.4780e-04 - mean_absolute_error: 0.0172
Epoch 210/10000
21/21 [==============================] - 0s 54us/step - loss: 4.4761e-04 - mean_absolute_error: 0.0172
Epoch 211/10000
21/21 [==============================] - 0s 81us/step - loss: 4.4742e-04 - mean_absolute_error: 0.0172
Epoch 212/10000
21/21 [==============================] - 0s 97us/step - loss: 4.4722e-04 - mean_absolute_error: 0.0172
Epoch 213/10000
21/21 [==============================] - 0s 48us/step - loss: 4.4703e-04 - mean_absolute_error: 0.0172
Epoch 214/10000
21/21 [==============================] - 0s 47us/step - loss: 4.4684e-04 - mean_absolute_error: 0.0172
Epoch 215/10000
21/21 [==============================] - 0s 56us/step - loss: 4.4664e-04 - mean_absolute_error: 0.0172
Epoch 216/10000
21/21 [==============================] - 0s 91us/step - loss: 4.4645e-04 - mean_absolute_error: 0.0172
Epoch 217/10000
21/21 [=========================

Epoch 278/10000
21/21 [==============================] - 0s 40us/step - loss: 4.3464e-04 - mean_absolute_error: 0.0169
Epoch 279/10000
21/21 [==============================] - 0s 55us/step - loss: 4.3445e-04 - mean_absolute_error: 0.0169
Epoch 280/10000
21/21 [==============================] - 0s 101us/step - loss: 4.3426e-04 - mean_absolute_error: 0.0169
Epoch 281/10000
21/21 [==============================] - 0s 57us/step - loss: 4.3407e-04 - mean_absolute_error: 0.0169
Epoch 282/10000
21/21 [==============================] - 0s 66us/step - loss: 4.3389e-04 - mean_absolute_error: 0.0169
Epoch 283/10000
21/21 [==============================] - 0s 60us/step - loss: 4.3370e-04 - mean_absolute_error: 0.0169
Epoch 284/10000
21/21 [==============================] - 0s 74us/step - loss: 4.3351e-04 - mean_absolute_error: 0.0169
Epoch 285/10000
21/21 [==============================] - 0s 85us/step - loss: 4.3332e-04 - mean_absolute_error: 0.0169
Epoch 286/10000
21/21 [========================

...

Epoch 9579/10000
21/21 [==============================] - 0s 47us/step - loss: 6.6160e-05 - mean_absolute_error: 0.0065
Epoch 9580/10000
21/21 [==============================] - 0s 43us/step - loss: 6.6237e-05 - mean_absolute_error: 0.0066
Epoch 9581/10000
21/21 [==============================] - 0s 52us/step - loss: 6.2436e-05 - mean_absolute_error: 0.0063
Epoch 9582/10000
21/21 [==============================] - 0s 45us/step - loss: 5.7348e-05 - mean_absolute_error: 0.0061
Epoch 9583/10000
21/21 [==============================] - 0s 74us/step - loss: 5.4082e-05 - mean_absolute_error: 0.0056
Epoch 9584/10000
21/21 [==============================] - 0s 45us/step - loss: 5.4086e-05 - mean_absolute_error: 0.0056
Epoch 9585/10000
21/21 [==============================] - 0s 49us/step - loss: 5.6445e-05 - mean_absolute_error: 0.0060
Epoch 9586/10000
21/21 [==============================] - 0s 49us/step - loss: 5.8923e-05 - mean_absolute_error: 0.0061
Epoch 9587/10000
21/21 [================

Epoch 9648/10000
21/21 [==============================] - 0s 39us/step - loss: 5.3270e-05 - mean_absolute_error: 0.0056
Epoch 9649/10000
21/21 [==============================] - 0s 41us/step - loss: 5.3267e-05 - mean_absolute_error: 0.0056
Epoch 9650/10000
21/21 [==============================] - 0s 50us/step - loss: 5.3262e-05 - mean_absolute_error: 0.0056
Epoch 9651/10000
21/21 [==============================] - 0s 61us/step - loss: 5.3256e-05 - mean_absolute_error: 0.0056
Epoch 9652/10000
21/21 [==============================] - 0s 44us/step - loss: 5.3247e-05 - mean_absolute_error: 0.0056
Epoch 9653/10000
21/21 [==============================] - 0s 44us/step - loss: 5.3237e-05 - mean_absolute_error: 0.0056
Epoch 9654/10000
21/21 [==============================] - 0s 71us/step - loss: 5.3227e-05 - mean_absolute_error: 0.0056
Epoch 9655/10000
21/21 [==============================] - 0s 53us/step - loss: 5.3219e-05 - mean_absolute_error: 0.0056
Epoch 9656/10000
21/21 [================

Epoch 9717/10000
21/21 [==============================] - 0s 47us/step - loss: 5.2814e-05 - mean_absolute_error: 0.0055
Epoch 9718/10000
21/21 [==============================] - 0s 94us/step - loss: 5.2808e-05 - mean_absolute_error: 0.0055
Epoch 9719/10000
21/21 [==============================] - 0s 64us/step - loss: 5.2803e-05 - mean_absolute_error: 0.0055
Epoch 9720/10000
21/21 [==============================] - 0s 118us/step - loss: 5.2797e-05 - mean_absolute_error: 0.0055
Epoch 9721/10000
21/21 [==============================] - 0s 62us/step - loss: 5.2793e-05 - mean_absolute_error: 0.0055
Epoch 9722/10000
21/21 [==============================] - 0s 52us/step - loss: 5.2788e-05 - mean_absolute_error: 0.0055
Epoch 9723/10000
21/21 [==============================] - 0s 72us/step - loss: 5.2785e-05 - mean_absolute_error: 0.0055
Epoch 9724/10000
21/21 [==============================] - 0s 66us/step - loss: 5.2782e-05 - mean_absolute_error: 0.0055
Epoch 9725/10000
21/21 [===============

Epoch 9786/10000
21/21 [==============================] - 0s 46us/step - loss: 5.2636e-05 - mean_absolute_error: 0.0055
Epoch 9787/10000
21/21 [==============================] - 0s 43us/step - loss: 5.2411e-05 - mean_absolute_error: 0.0055
Epoch 9788/10000
21/21 [==============================] - 0s 62us/step - loss: 5.2360e-05 - mean_absolute_error: 0.0055
Epoch 9789/10000
21/21 [==============================] - 0s 44us/step - loss: 5.2462e-05 - mean_absolute_error: 0.0055
Epoch 9790/10000
21/21 [==============================] - 0s 45us/step - loss: 5.2606e-05 - mean_absolute_error: 0.0055
Epoch 9791/10000
21/21 [==============================] - 0s 60us/step - loss: 5.2681e-05 - mean_absolute_error: 0.0056
Epoch 9792/10000
21/21 [==============================] - 0s 54us/step - loss: 5.2639e-05 - mean_absolute_error: 0.0055
Epoch 9793/10000
21/21 [==============================] - 0s 46us/step - loss: 5.2514e-05 - mean_absolute_error: 0.0055
Epoch 9794/10000
21/21 [================

Epoch 9855/10000
21/21 [==============================] - 0s 49us/step - loss: 5.1937e-05 - mean_absolute_error: 0.0055
Epoch 9856/10000
21/21 [==============================] - 0s 53us/step - loss: 5.1931e-05 - mean_absolute_error: 0.0055
Epoch 9857/10000
21/21 [==============================] - 0s 58us/step - loss: 5.1925e-05 - mean_absolute_error: 0.0055
Epoch 9858/10000
21/21 [==============================] - 0s 52us/step - loss: 5.1918e-05 - mean_absolute_error: 0.0055
Epoch 9859/10000
21/21 [==============================] - 0s 47us/step - loss: 5.1912e-05 - mean_absolute_error: 0.0055
Epoch 9860/10000
21/21 [==============================] - 0s 51us/step - loss: 5.1905e-05 - mean_absolute_error: 0.0055
Epoch 9861/10000
21/21 [==============================] - 0s 45us/step - loss: 5.1899e-05 - mean_absolute_error: 0.0055
Epoch 9862/10000
21/21 [==============================] - 0s 44us/step - loss: 5.1893e-05 - mean_absolute_error: 0.0055
Epoch 9863/10000
21/21 [================

### Predict

In [39]:
dfp_result = dfp_model.predict(top10_bot10)

### Visualize

In [45]:
predicted_spy_and_original = np.array((dfp_result, SPMC_SPY_relative_price_benchmark.values))
predicted_spy_and_original = pd.DataFrame(predicted_spy_and_original.reshape(21,2))
predicted_spy_and_original.columns = ['Prediction', 'Original']
predicted_spy_and_original.index = SPMC_SPY_relative_price_benchmark.index
iplot(predicted_spy_and_original.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Relative Price',title='Predicted SPY vs SPY'))


### Percentage error for each day's relative price

In [46]:
pd.DataFrame(abs(dfp_result - SPMC_SPY_relative_price_benchmark)/SPMC_SPY_relative_price_benchmark) * 100

,SPY
Date,
2008-09-02,3.412771
2008-09-03,3.935046
2008-09-04,0.659792
2008-09-05,0.729642
2008-09-08,4.121817
2008-09-09,1.897816
2008-09-10,1.773541
2008-09-11,1.460484
2008-09-12,0.984661
